In [1]:
import torch

from torch_geometric.utils import to_dense_adj
from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import DataLoader, random_split


In [2]:
from helpers import CVFConfigDataset

In [ ]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [4]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_3_config_rank_dataset.csv", "graph_3_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
# )
# dataset = CVFConfigDataset(
#     "graph_6_config_rank_dataset.csv", "graph_6_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_6b_config_rank_dataset.csv", "graph_6b_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_7_config_rank_dataset.csv", "graph_7_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_8_config_rank_dataset.csv", "graph_8_edge_index.json", 7
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n5_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n6_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n6_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n7_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n8_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n8_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )

dataset = CVFConfigDataset(
    "implicit_graph_n10_config_rank_dataset.csv",
    "implicit_graph_n10_edge_index.json",
    3
)

train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=102400, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0).to(device)
adjacency

tensor([[[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 1.]]], device='cuda:0')

In [7]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        # h = torch.sigmoid(h)
        # h = h.squeeze(-1)
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.CrossEntropyLoss()
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        for epoch in range(1, epochs + 1):
            self.train()
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                # y = y.reshape(-1, 1)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, adjacency)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)


In [ ]:
gnn = VanillaGNN(dataset.num_classes, 256, 1).to(device)
print(gnn)

gnn.fit(train_loader, epochs=100)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=3, out_features=256, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=256, out_features=256, bias=False)
  )
  (out): Linear(in_features=256, out_features=1, bias=True)
)
Loss: tensor(4673.3599, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(3229.7095, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1771.0663, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(649.8027, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(852.4231, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1538.6804, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1237.9885, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(718.0177, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(505.8555, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(607.6956, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(790.2056, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(886.7863

In [ ]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, adjacency)
    # predicted = predicted.argmax(dim=1)
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    

print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")

Total matched 305 out of 14763 | Accuracy 2.066 %
